In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

- 부동산 평당분양가격 df 활용

# 1. 데이터셋

In [3]:
import pandas as pd
df = pd.read_csv('../01_python/data/ch13_df(결측치중위값으로대체).csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역명     2176 non-null   object 
 1   평당분양가격  2176 non-null   float64
 2   연도      2176 non-null   int64  
 3   월       2176 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 68.1+ KB


In [5]:
df.head()

,지역명,평당분양가격,연도,월
0,서울,18189.0,2013,12
1,부산,8111.0,2013,12
2,대구,8080.0,2013,12
3,인천,10204.0,2013,12
4,광주,6098.0,2013,12


- 지역명2 : 지역명 필드를 라벨인코딩하여 추가
- 독립변수 : 지역명2, 연도, 월
- 타깃변수 : 평당분양가격
- 독립변수와 타깃변수의 scaling
    - 정규화, 표준화 모두 시도 후 나은 결과 채택
    - MinMaxScaler 정규화 작업 후 : 지역명2m, 연도m, 월m
    - StandardScaler 표준화 작업 후 : 지역명2s, 연도s, 월s → 지역명, 연도, 월, 지역명2, 지역명2m, 연도m, 월m, 지역명2s, 연도s, 월s

# 2. 지역명 라벨인코딩
지역명2

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# le.fit(df['지역명'])
# le.transform(df['지역명'])
df['지역명2'] = le.fit_transform(df['지역명']) #문자데이터를 오름차순 정렬 후 숫자로 변환
df.head()

,지역명,평당분양가격,연도,월,지역명2
0,서울,18189.0,2013,12,8
1,부산,8111.0,2013,12,7
2,대구,8080.0,2013,12,5
3,인천,10204.0,2013,12,11
4,광주,6098.0,2013,12,4


In [17]:
df['지역명'].unique()

array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '세종', '강원', '충북',
       '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object)

# 3. normalization 스케일 조정
MinMaxScaler

In [20]:
X_data = df[['지역명2', '연도', '월']].values #numpy 배열로 추출한 독립 변수
y_data = df[['평당분양가격']].to_numpy() #values나 to_numpy()를 이용해서 데이터프레임을 numpy 배열로 변환
X_data.shape, y_data.shape

((2176, 3), (2176, 1))

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
df[['지역명2m', '연도m', '월m']] = scaler_X.fit_transform(X_data)
df[['평당분양가격m']] = scaler_y.fit_transform(y_data)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757


# 4. standardization 스케일 조정
StandardScaler

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
df[['지역명2s', '연도s', '월s']] = scaler_x.fit_transform(X_data)
df[['평당분양가격s']] = scaler_y.fit_transform(y_data)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203


# 5. 지역명 원핫인코딩
- 강원 0 → 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
- 서울 8 → 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0

In [29]:
loc = df.loc[:16, ['지역명', '지역명2']].sort_values(by='지역명2')['지역명'].tolist()
print(loc) #원핫인코딩 후 열이름

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [32]:
import numpy as np
from tensorflow.keras.utils import to_categorical # 분류분석 시 원핫인코딩(추천)
temp1 = to_categorical(df['지역명2']) 
#to_categorical: numpy array. 최대 정수가 16이면 0~16 카테고리로 원핫인코딩. 매개변수에 숫자만 가능
temp2 = pd.get_dummies(df['지역명']).values
#pd.get_dummies: dataframe. 정수가 곧 카테고리가 돼서 원핫인코딩. 매개변수에 문자 필드 가능
np.all(temp1 == temp2)

True

In [34]:
# 원핫인코딩 방법 1 to_categorical
df[loc] = to_categorical(df['지역명2'])#.astype(np.int16)
pd.options.display.max_columns = 30
df[loc].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   강원      2176 non-null   float32
 1   경기      2176 non-null   float32
 2   경남      2176 non-null   float32
 3   경북      2176 non-null   float32
 4   광주      2176 non-null   float32
 5   대구      2176 non-null   float32
 6   대전      2176 non-null   float32
 7   부산      2176 non-null   float32
 8   서울      2176 non-null   float32
 9   세종      2176 non-null   float32
 10  울산      2176 non-null   float32
 11  인천      2176 non-null   float32
 12  전남      2176 non-null   float32
 13  전북      2176 non-null   float32
 14  제주      2176 non-null   float32
 15  충남      2176 non-null   float32
 16  충북      2176 non-null   float32
dtypes: float32(17)
memory usage: 144.6 KB


In [35]:
df[loc] = df[loc].astype(np.int16)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
# 원핫인코딩 방법 2 pd.get_dummies
df[loc] = pd.get_dummies(df['지역명2'])
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
# 원핫인코딩 데이터 지우기
df = df.drop(loc, axis=1)
df.head(1)

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,18189.0,2013,12,8,0.5,0.0,1.0,0.328198,0.0,-1.875367,1.62196,1.168591


In [40]:
# 원핫인코딩 방법 3 concat
df = pd.concat([df, pd.get_dummies(df['지역명'])], axis=1)
df.sample()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
1259,인천,14058.0,2020,3,11,0.6875,0.636364,0.181818,0.220425,0.612372,0.377084,-0.984034,0.391045,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
